In [1]:
%use dataframe

In [10]:
import kotlin.math.ln
import kotlin.math.min
import kotlin.math.exp
import kotlin.random.Random

In [3]:
val rand = Random()

Line_5.jupyter.kts (1:12 - 20) Cannot create an instance of an abstract class

In [53]:
val df = dataFrameOf(
    "X" to List(100) { if (it<50) 1 else 1 }
).add("y") {
    val p= 1/(1+exp(-0.0*"X"<Int>().toFloat()))
    if (Random.nextFloat()<p) 1 else 0
}
df.head(10)

X,y
1,1
1,1
1,0
1,1
1,1
1,0
1,1
1,0
1,0
1,0


In [57]:
df.writeCSV("testdata.csv")

In [5]:
val df2=df.add("time") {
      min(eventtime,censur) 
}.add("event") { eventtime < censur }
.remove {eventtime and censur }

In [6]:
val totals = df2.groupBy { group }.aggregate { count() into "total" }

In [7]:
totals

group,total
0,50
1,50


In [19]:
totals.filter { group == 1 }.total.toList()

[50]

In [68]:
val theta: (beta: DoubleArray, X: DoubleArray) -> Double =  { beta,X ->  beta.foldIndexed(initial=0.0, operation = {idx, acc, d -> acc+d*X[idx]}) }
val dtheta: (beta: DoubleArray, X: DoubleArray) -> DoubleArray =  { beta,X ->  DoubleArray(X.size) { 1.0 } }
val ddtheta: (beta: DoubleArray, X: DoubleArray) -> DoubleArray =  { beta,X ->  DoubleArray(X.size) { 0.0 } }

val b: (theta: Double) -> Double =  { theta ->  ln(1+exp(theta)) }
val db: (theta: Double) -> Double =  { theta ->  exp(theta)/(1+exp(theta)) }
val ddb: (theta: Double) -> Double =  { theta ->  exp(theta)/((1+exp(theta)).pow(2.0)) }

In [41]:
theta(0.0)

0.0

In [46]:
df.sumOf { (y.toDouble() -db(theta(0.0))) *dtheta(0.0) }

23.0

In [48]:
df.sumOf { ( -ddb(theta(0.0))) *(dtheta(0.0).pow(2.0)) }

-25.0

In [59]:
var beta=0.0
repeat(15) {
    beta = beta - df.sumOf { (y.toDouble() -db(theta(beta))) *dtheta(beta) }/
    (df.sumOf { (y.toDouble() -db(theta(beta))) *ddtheta(beta) } + df.sumOf { ( -ddb(theta(beta))) *(dtheta(beta).pow(2.0)) })
    println(beta)
}

-0.16
-0.1603426453855212
-0.1603426500751791
-0.1603426500751792
-0.16034265007517912
-0.16034265007517923
-0.16034265007517914
-0.16034265007517925
-0.16034265007517917
-0.16034265007517928
-0.1603426500751792
-0.16034265007517912
-0.16034265007517923
-0.16034265007517914
-0.16034265007517925
